In [11]:
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDClassifier

In [24]:
train_data = pd.read_csv('./train.csv')
train_ans = pd.read_csv('./train_answers.csv')

In [31]:
train_tort_data = train_data.iloc[:, 4:44]
train_tort_ans = train_ans.iloc[:,1]

x = train_tort_data.to_numpy()
y = np.repeat(train_tort_ans.to_numpy(), 4)

In [32]:
print(np.shape(x))
print(np.shape(y))

(1128, 40)
(1128,)


In [33]:
model = SGDClassifier()
model.fit(X=x, y=y)

SGDClassifier()

In [34]:
answers = model.predict(X=x)

In [36]:
print((answers != y).sum())

547
